In [ ]:
import time
import random
import pandas as pd
import requests
import json
from datetime import datetime

# Function to generate simulated sensor data
def generate_sensor_data(output_file="sensor_data.csv", iterations=10, interval=2):
    # Open the file in append mode
    with open(output_file, "a") as f:
        # Write a header if the file is empty
        if f.tell() == 0:
            f.write("timestamp,temperature,pressure,humidity\n")
        
        for _ in range(iterations):  # Limit the number of iterations
            data = {
                "timestamp": pd.Timestamp.now(),
                "temperature": round(random.uniform(20.0, 100.0), 2),
                "pressure": round(random.uniform(1.0, 10.0), 2),
                "humidity": round(random.uniform(30.0, 80.0), 2),
            }
            print(data)  # Print to console for monitoring
            f.write(f'{data["timestamp"]},{data["temperature"]},{data["pressure"]},{data["humidity"]}\n')
            time.sleep(interval)  # Wait for the specified interval before generating next data point

# Run the data generator
if __name__ == "__main__":
    generate_sensor_data(output_file="sensor_data.csv", iterations=50, interval=2)

# Configuration and connection with pivison
pi_web_api_url = "http://<PI_SERVER_NAME>/piwebapi"
pi_point = "<PI_POINT_NAME>"  # Example: "Sensor1_Temperature"
headers = {"Content-Type": "application/json"}

# Function to send data to PI Web API
def send_to_pi(data):
    url = f"{pi_web_api_url}/streams/{pi_point}/value"
    payload = {
        "Timestamp": data["timestamp"],
        "Value": data["temperature"]
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    if response.status_code == 204:
        print("Data sent successfully!")
    else:
        print(f"Failed to send data: {response.text}")

# Example of sending data
data = {
    "timestamp": datetime.utcnow().isoformat(),
    "temperature": 25.5  # Replace with your actual data
}
send_to_pi(data)



{'timestamp': Timestamp('2024-12-18 17:27:46.042925'), 'temperature': 87.72, 'pressure': 3.44, 'humidity': 47.83}
{'timestamp': Timestamp('2024-12-18 17:27:48.057625'), 'temperature': 59.82, 'pressure': 4.93, 'humidity': 43.87}
{'timestamp': Timestamp('2024-12-18 17:27:50.059358'), 'temperature': 56.59, 'pressure': 9.32, 'humidity': 38.9}
{'timestamp': Timestamp('2024-12-18 17:27:52.060747'), 'temperature': 36.27, 'pressure': 9.14, 'humidity': 32.22}
{'timestamp': Timestamp('2024-12-18 17:27:54.065057'), 'temperature': 24.57, 'pressure': 2.15, 'humidity': 39.96}
{'timestamp': Timestamp('2024-12-18 17:27:56.069893'), 'temperature': 68.09, 'pressure': 1.28, 'humidity': 47.4}
{'timestamp': Timestamp('2024-12-18 17:27:58.071872'), 'temperature': 77.86, 'pressure': 2.34, 'humidity': 45.5}
{'timestamp': Timestamp('2024-12-18 17:28:00.074942'), 'temperature': 89.33, 'pressure': 3.77, 'humidity': 65.16}
{'timestamp': Timestamp('2024-12-18 17:28:02.079910'), 'temperature': 88.05, 'pressure': 6.

In [3]:
import pandas as pd

# Load the CSV file
try:
    data = pd.read_csv("sensor_data.csv")
    print("CSV loaded successfully!")
except FileNotFoundError:
    print("File 'sensor_data.csv' not found. Please check the file path.")
    exit()
except Exception as e:
    print(f"An error occurred while loading the file: {e}")
    exit()

# Debug: Print column names and first few rows
print("Columns in the dataset:", data.columns)
print("First few rows of the dataset:")
print(data.head())

# Check if 'timestamp' column exists
if 'timestamp' not in data.columns:
    print("The 'timestamp' column is missing from the dataset.")
    print("Columns present in the dataset:", data.columns)
    exit()

# Convert the 'timestamp' column to datetime format
try:
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    print("Timestamp conversion successful!")
except Exception as e:
    print(f"Error converting 'timestamp' to datetime: {e}")
    exit()

# Add a 'time_delta' column to calculate time differences
try:
    data['time_delta'] = (data['timestamp'] - data['timestamp'].min()).dt.total_seconds()
    print("Time delta calculated successfully!")
except Exception as e:
    print(f"Error calculating time delta: {e}")
    exit()

# Print the cleaned dataset
print("Cleaned dataset:")
print(data.head())

# Save the cleaned dataset to a new CSV file
try:
    data.to_csv("cleaned_sensor_data.csv", index=False)
    print("Cleaned data saved to 'cleaned_sensor_data.csv'.")
except Exception as e:
    print(f"Error saving the cleaned data: {e}")


CSV loaded successfully!
Columns in the dataset: Index(['2024-12-18 17:07:21.614736', '70.8', '4.65', '69.83'], dtype='object')
First few rows of the dataset:
   2024-12-18 17:07:21.614736   70.8  4.65  69.83
0  2024-12-18 17:07:23.622996  41.44  8.00  75.06
1  2024-12-18 17:07:25.629292  89.71  3.69  79.97
2  2024-12-18 17:07:27.634499  92.84  8.51  33.06
3  2024-12-18 17:07:29.640313  98.05  4.27  76.20
4  2024-12-18 17:07:31.644340  47.32  4.58  57.78
The 'timestamp' column is missing from the dataset.
Columns present in the dataset: Index(['2024-12-18 17:07:21.614736', '70.8', '4.65', '69.83'], dtype='object')
Error converting 'timestamp' to datetime: 'timestamp'
Error calculating time delta: 'timestamp'
Cleaned dataset:
   2024-12-18 17:07:21.614736   70.8  4.65  69.83
0  2024-12-18 17:07:23.622996  41.44  8.00  75.06
1  2024-12-18 17:07:25.629292  89.71  3.69  79.97
2  2024-12-18 17:07:27.634499  92.84  8.51  33.06
3  2024-12-18 17:07:29.640313  98.05  4.27  76.20
4  2024-12-18 1

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Load cleaned data
data = pd.read_csv("cleaned_sensor_data.csv")
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['time_delta'] = (data['timestamp'] - data['timestamp'].min()).dt.total_seconds()

# Train a model on temperature data
X = data['time_delta'].values.reshape(-1, 1)
y = data['temperature'].values
model = LinearRegression()
model.fit(X, y)

# Predict future temperatures
future_time = np.array([data['time_delta'].max() + 3600]).reshape(-1, 1)
future_temp = model.predict(future_time)
print(f"Predicted temperature in one hour: {future_temp[0]:.2f}")
